In [2]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/통신망해커톤/Q1_data')
os.getcwd()

In [7]:
# 데이터 불러오기
# 경로 바꿔서 사용할 것
train = pd.read_csv('Q1_train.csv')
test = pd.read_csv('Q1_test.csv')

In [8]:
test.drop('Unnamed: 0', axis=1, inplace=True)
test.head(1)

,datetime,ru_id,scgfail,scgfailratio,erabaddatt,erabaddsucc,endcaddatt,endcaddsucc,endcmodbymenbatt,endcmodbymenbsucc,...,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,numrar,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,2023-05-19 00:00:00,BaseStationJ,0,0.0,5,5,5,5,0,0,...,412.0,13.3,22.4,73.0,73.0,7.0,1401.0,-93.0,9.8,0


#전처리

In [9]:
train.isna().sum()
# 결측치 9개 전체 데이터대비 매우 적어서 드랍하기로 결정

datetime                              0
ru_id                                 0
scgfail                               0
scgfailratio                          0
erabaddatt                            0
erabaddsucc                           0
endcaddatt                            0
endcaddsucc                           0
endcmodbymenbatt                      0
endcmodbymenbsucc                     0
endcmodbysgnbatt                      0
endcmodbysgnbsucc                     0
connestabatt                          0
connestabsucc                         0
redirectiontolte_coverageout          0
redirectiontolte_epsfallback          0
redirectiontolte_emergencyfallback    0
handoveratt                           0
handoversucc                          0
reestabatt                            0
reestabsucc                           0
rlculbyte                             9
rlcdlbyte                             9
totprbulavg                           9
totprbdlavg                           9


In [10]:
train.dropna(inplace=True)
train.reset_index(drop=True, inplace=True)
len(train)

137436

In [11]:
# scgtry 변수 생성
train['scgtry'] = 1/(train['scgfailratio']/100/train['scgfail'])
train['scgtry'].fillna(0,inplace=True)
test['scgtry'] = 1/(test['scgfailratio']/100/test['scgfail'])
test['scgtry'].fillna(0,inplace=True)
train.loc[train['scgtry']==float("inf"), 'scgtry'] = 0
test.loc[test['scgtry']==float("inf"), 'scgtry'] = 0

In [15]:
# 기초 통계량
train['scgtry'].describe()

count    137436.000000
mean          8.214608
std          21.854702
min           0.000000
25%           0.000000
50%           0.000000
75%           3.003003
max         833.333333
Name: scgtry, dtype: float64

## 파생변수

In [13]:
# ratio 파생변수 생성
train['erabadd_r'] = train['erabaddsucc'] / train['erabaddatt']
train['endcadd_r'] = train['endcaddsucc'] / train['endcaddatt']
train['endcmodbymenb_r'] = train['endcmodbymenbsucc'] / train['endcmodbymenbatt']
train['endcmodbysgnb_r'] = train['endcmodbysgnbsucc'] / train['endcmodbysgnbatt']
train['connestab_r'] = train['connestabsucc'] / train['connestabatt']
train['handover_r'] = train['handoversucc'] / train['handoveratt']
train['reestab_r'] = train['reestabsucc'] / train['reestabatt']

test['erabadd_r'] = test['erabaddsucc'] / test['erabaddatt']
test['endcadd_r'] = test['endcaddsucc'] / test['endcaddatt']
test['endcmodbymenb_r'] = test['endcmodbymenbsucc'] / test['endcmodbymenbatt']
test['endcmodbysgnb_r'] = test['endcmodbysgnbsucc'] / test['endcmodbysgnbatt']
test['connestab_r'] = test['connestabsucc'] / test['connestabatt']
test['handover_r'] = test['handoversucc'] / test['handoveratt']
test['reestab_r'] = test['reestabsucc'] / test['reestabatt']

In [16]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [17]:
# station 별로 분리 하기 위해 새로운 데이터 파일 불러오기
raw = pd.read_csv('/content/drive/MyDrive/통신망해커톤/Q1_data/Q1_train.csv')
raw.dropna(inplace=True)
raw.reset_index(drop=True, inplace=True)

In [19]:
# station 별로 분리
idx_i = list(raw.loc[raw['ru_id']=='BaseStationI'].index)
idx_d = list(raw.loc[raw['ru_id']=='BaseStationD'].index)
idx_c = list(raw.loc[raw['ru_id']=='BaseStationC'].index)
idx_a = list(raw.loc[raw['ru_id']=='BaseStationA'].index)
idx_h = list(raw.loc[raw['ru_id']=='BaseStationH'].index)
idx_g = list(raw.loc[raw['ru_id']=='BaseStationG'].index)
idx_e = list(raw.loc[raw['ru_id']=='BaseStationE'].index)
idx_f = list(raw.loc[raw['ru_id']=='BaseStationF'].index)

df_i = train.loc[idx_i]
df_d = train.loc[idx_d]
df_c = train.loc[idx_c]
df_a = train.loc[idx_a]
df_h = train.loc[idx_h]
df_g = train.loc[idx_g]
df_e = train.loc[idx_e]
df_f = train.loc[idx_f]

dflist = [df_i, df_d, df_c, df_a, df_h, df_g, df_e, df_f]

In [20]:
# 1시점 차분 값
for df in dflist:
  df['rlculbyte_diff'] = df['rlculbyte'].diff()
  df['rlcdlbyte_diff'] = df['rlcdlbyte'].diff()
  df['totprbulavg_diff'] = df['totprbulavg'].diff()
  df['totprbdlavg_diff'] = df['totprbdlavg'].diff()
  df['dlreceivedriavg_diff'] = df['dlreceivedriavg'].diff()
  df['dltransmittedmcsavg_diff'] = df['dltransmittedmcsavg'].diff()
  df['airmaculbyte_diff'] = df['airmaculbyte'].diff()
  df['airmacdlbyte_diff'] = df['airmacdlbyte'].diff()
  df['attpaging_diff'] = df['attpaging'].diff()
  df['rssipathavg_diff'] = df['rssipathavg'].diff()
  df['endcrelbymenb_diff'] = df['endcrelbymenb'].diff()
  df['nummsg3_diff'] = df['nummsg3'].diff()
  df['dlreceivedcqiavg_diff'] = df['dlreceivedcqiavg'].diff()
  df['rachpreamblea_diff'] = df['rachpreamblea'].diff()

In [21]:
dflist = [df_i, df_d, df_c, df_a, df_h, df_g, df_e, df_f]

In [23]:
# 원본 데이터에 추가
train['rlculbyte_diff'] = np.nan
train['rlcdlbyte_diff'] = np.nan
train['totprbulavg_diff'] = np.nan
train['totprbdlavg_diff'] = np.nan
train['dlreceivedriavg_diff'] = np.nan
train['dltransmittedmcsavg_diff'] = np.nan
train['airmaculbyte_diff'] = np.nan
train['airmacdlbyte_diff'] = np.nan
train['attpaging_diff'] = np.nan
train['rssipathavg_diff'] = np.nan
train['endcrelbymenb_diff'] = np.nan
train['nummsg3_diff'] = np.nan
train['dlreceivedcqiavg_diff'] = np.nan
train['rachpreamblea_diff'] = np.nan

In [24]:
cols = list(df.iloc[:,-14:].columns)
cols

['rlculbyte_diff',
 'rlcdlbyte_diff',
 'totprbulavg_diff',
 'totprbdlavg_diff',
 'dlreceivedriavg_diff',
 'dltransmittedmcsavg_diff',
 'airmaculbyte_diff',
 'airmacdlbyte_diff',
 'attpaging_diff',
 'rssipathavg_diff',
 'endcrelbymenb_diff',
 'nummsg3_diff',
 'dlreceivedcqiavg_diff',
 'rachpreamblea_diff']

In [25]:
for df in dflist:
  train.loc[list(df.index), cols] = df.loc[list(df.index), cols].values

In [26]:
# 원본데이터에 잘 합쳐졌나 확인
pd.set_option('display.max_rows', None)
train.isnull().sum()

datetime                              0
ru_id                                 0
scgfail                               0
scgfailratio                          0
erabaddatt                            0
erabaddsucc                           0
endcaddatt                            0
endcaddsucc                           0
endcmodbymenbatt                      0
endcmodbymenbsucc                     0
endcmodbysgnbatt                      0
endcmodbysgnbsucc                     0
connestabatt                          0
connestabsucc                         0
redirectiontolte_coverageout          0
redirectiontolte_epsfallback          0
redirectiontolte_emergencyfallback    0
handoveratt                           0
handoversucc                          0
reestabatt                            0
reestabsucc                           0
rlculbyte                             0
rlcdlbyte                             0
totprbulavg                           0
totprbdlavg                           0


In [27]:
train.dropna(inplace=True)
train.reset_index(drop=True, inplace=True)
len(train)

137428

In [28]:
#y변수를 가장 마지막으로 (열 순서 변경)
y = train['uenomax']
train.drop('uenomax', axis=1, inplace=True)
train['uenomax'] = y

In [29]:
# test에 대해서도 동일하게 전처리
raw = pd.read_csv('/content/drive/MyDrive/통신망해커톤/Q1_data/Q1_test.csv')
raw.dropna(inplace=True)
raw.reset_index(drop=True, inplace=True)

In [30]:
idx_b = list(raw.loc[raw['ru_id']=='BaseStationB'].index)
idx_j = list(raw.loc[raw['ru_id']=='BaseStationJ'].index)

df_b = test.loc[idx_b]
df_j = test.loc[idx_j]

dflist = [df_b, df_j]

In [31]:
for df in dflist:
  df['rlculbyte_diff'] = df['rlculbyte'].diff()
  df['rlcdlbyte_diff'] = df['rlcdlbyte'].diff()
  df['totprbulavg_diff'] = df['totprbulavg'].diff()
  df['totprbdlavg_diff'] = df['totprbdlavg'].diff()
  df['dlreceivedriavg_diff'] = df['dlreceivedriavg'].diff()
  df['dltransmittedmcsavg_diff'] = df['dltransmittedmcsavg'].diff()
  df['airmaculbyte_diff'] = df['airmaculbyte'].diff()
  df['airmacdlbyte_diff'] = df['airmacdlbyte'].diff()
  df['attpaging_diff'] = df['attpaging'].diff()
  df['rssipathavg_diff'] = df['rssipathavg'].diff()
  df['endcrelbymenb_diff'] = df['endcrelbymenb'].diff()
  df['nummsg3_diff'] = df['nummsg3'].diff()
  df['dlreceivedcqiavg_diff'] = df['dlreceivedcqiavg'].diff()
  df['rachpreamblea_diff'] = df['rachpreamblea'].diff()

In [32]:
df_b.fillna(df_b.mean(), inplace=True)
df_j.fillna(df_j.mean(), inplace=True)

<ipython-input-32-0afa24fc0bca>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_b.fillna(df_b.mean(), inplace=True)
<ipython-input-32-0afa24fc0bca>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_j.fillna(df_j.mean(), inplace=True)


In [33]:
dflist = [df_b, df_j]

In [34]:
test['rlculbyte_diff'] = np.nan
test['rlcdlbyte_diff'] = np.nan
test['totprbulavg_diff'] = np.nan
test['totprbdlavg_diff'] = np.nan
test['dlreceivedriavg_diff'] = np.nan
test['dltransmittedmcsavg_diff'] = np.nan
test['airmaculbyte_diff'] = np.nan
test['airmacdlbyte_diff'] = np.nan
test['attpaging_diff'] = np.nan
test['rssipathavg_diff'] = np.nan
test['endcrelbymenb_diff'] = np.nan
test['nummsg3_diff'] = np.nan
test['dlreceivedcqiavg_diff'] = np.nan
test['rachpreamblea_diff'] = np.nan

In [35]:
for df in dflist:
  test.loc[list(df.index), cols] = df.loc[list(df.index), cols].values

In [36]:
test.isnull().sum()

datetime                              0
ru_id                                 0
scgfail                               0
scgfailratio                          0
erabaddatt                            0
erabaddsucc                           0
endcaddatt                            0
endcaddsucc                           0
endcmodbymenbatt                      0
endcmodbymenbsucc                     0
endcmodbysgnbatt                      0
endcmodbysgnbsucc                     0
connestabatt                          0
connestabsucc                         0
redirectiontolte_coverageout          0
redirectiontolte_epsfallback          0
redirectiontolte_emergencyfallback    0
handoveratt                           0
handoversucc                          0
reestabatt                            0
reestabsucc                           0
rlculbyte                             0
rlcdlbyte                             0
totprbulavg                           0
totprbdlavg                           0


##왜도 조정

In [37]:
# X변수 왜도 확인
# 대체적으로 왜도가 매우높음
for col in train.iloc[:, 2:].columns:
  skewness = train[col].skew()
  print(col, ':', skewness)

scgfail : 39.95070162915557
scgfailratio : 8.743806184187866
erabaddatt : 6.8765629532018275
erabaddsucc : 6.878388415963638
endcaddatt : 8.109116613906583
endcaddsucc : 8.109682275794837
endcmodbymenbatt : 12.128443620608161
endcmodbymenbsucc : 12.133968909257673
endcmodbysgnbatt : 7.915806812294115
endcmodbysgnbsucc : 7.9577019202422985
connestabatt : 9.510942974943191
connestabsucc : 9.506677076318132
redirectiontolte_coverageout : 7.918968643345804
redirectiontolte_epsfallback : 6.747821810204825
redirectiontolte_emergencyfallback : 185.3503438846847
handoveratt : 2.9877801056549766
handoversucc : 2.984256633404174
reestabatt : 42.6182560965328
reestabsucc : 45.44777521070747
rlculbyte : 29.90212227058559
rlcdlbyte : 34.97781321914227
totprbulavg : 26.985944092590675
totprbdlavg : 11.671348576266618
dlreceivedriavg : -0.4221745655997072
dltransmittedmcsavg : 0.4928600272832001
airmaculbyte : 25.101807677728235
airmacdlbyte : 30.787141131074108
bler_ul : 10.694957300068879
bler_dl :

In [38]:
from sklearn.preprocessing import PowerTransformer

#X변수만 왜도 조정
for col in train.iloc[:, 2:-1].columns:
  skewness = train[col].skew()
  if abs(skewness) >= 2: #왜도의 절댓값이 2 이상이면 변환
    pt = PowerTransformer()
    pt.fit(train[[col]])
    train[col] = pt.transform(train[[col]]) #디폴트로 여존슨변환
    test[col] = pt.transform(test[[col]])

In [39]:
# X변수 왜도 변환 확인
for col in train.iloc[:, 2:].columns:
  skewness = train[col].skew()
  print(col, ':', skewness)

scgfail : 1.114337621551466
scgfailratio : 1.1328575648055426
erabaddatt : 0.020799351786181877
erabaddsucc : 0.020804041735144607
endcaddatt : 0.029399968185337542
endcaddsucc : 0.029396609868653046
endcmodbymenbatt : 0.09921666486425791
endcmodbymenbsucc : 0.09944064453273016
endcmodbysgnbatt : 0.00884369296191755
endcmodbysgnbsucc : 0.009302526774277767
connestabatt : 0.026415785404855202
connestabsucc : 0.026576081866399533
redirectiontolte_coverageout : 1.9476369015539894
redirectiontolte_epsfallback : 3.127668240261311
redirectiontolte_emergencyfallback : 185.35034388468492
handoveratt : 0.3510113455744262
handoversucc : 0.35160143355633294
reestabatt : 7.247312094977947
reestabsucc : 7.481654037620226
rlculbyte : -0.03125691008447746
rlcdlbyte : -0.19874225459538397
totprbulavg : 0.38095416678384514
totprbdlavg : -0.20756533654669232
dlreceivedriavg : -0.4221745655997072
dltransmittedmcsavg : 0.4928600272832001
airmaculbyte : 0.013634649701374632
airmacdlbyte : -0.15428616397056

##시간 전처리

In [40]:
# 시간을 각각 month, date, hour, minute으로 변환
# 주말(공휴일) 여부 추가
train['datetime'] = train['datetime'].astype(str)
test['datetime'] = test['datetime'].astype(str)

from datetime import datetime

def remove_timezone_from_date(date_string):
    return date_string[:-6]

newtime = [remove_timezone_from_date(date_str) for date_str in train['datetime']]
train['datetime'] = newtime
train['datetime'] = pd.to_datetime(train['datetime'])
train['month'] = train['datetime'].dt.month
train['date'] = train['datetime'].dt.day
train['time_h'] = train['datetime'].dt.hour
train['time_m'] = train['datetime'].dt.minute
yoil = train['datetime'].dt.weekday
train['holiday'] =  yoil.apply(lambda x: 1 if x == 5 else (1 if x == 6 else 0))
train.drop('datetime', axis=1, inplace=True)

newtime = [remove_timezone_from_date(date_str) for date_str in test['datetime']]
test['datetime'] = newtime
test['datetime'] = pd.to_datetime(test['datetime'])
test['month'] = test['datetime'].dt.month
test['date'] = test['datetime'].dt.day
test['time_h'] = test['datetime'].dt.hour
test['time_m'] = test['datetime'].dt.minute
yoil = test['datetime'].dt.weekday
test['holiday'] =  yoil.apply(lambda x: 1 if x == 5 else (1 if x == 6 else 0))
test.drop('datetime', axis=1, inplace=True)

In [43]:
# train set에 공휴일 추가
train.loc[train['month']==5].loc[train['date']==29, 'holiday'] = 1
train.loc[train['month']==6].loc[train['date']==6, 'holiday'] = 1

In [44]:
# test set에 공휴일 추가
test.loc[test['month']==5].loc[test['date']==29, 'holiday'] = 1
test.loc[test['month']==6].loc[test['date']==6, 'holiday'] = 1

# 0개수 카운트

In [46]:
# 행의 0 갯수 추가
# train 에 진행
df = train.copy()
df = df.drop(['time_h', 'holiday'],axis=1)
df.dropna(inplace=True)
zero_counts = df.apply(lambda row: row.value_counts().get(0, 0), axis=1)
train['zero'] = pd.DataFrame(zero_counts)

In [47]:
# test에 동일하게 진행
df = test.copy()
df = df.drop(['time_h', 'holiday'],axis=1)
df.dropna(inplace=True)
zero_counts = df.apply(lambda row: row.value_counts().get(0, 0), axis=1)
test['zero'] = pd.DataFrame(zero_counts)

#필요없는 변수 드랍

In [48]:
from scipy.stats import chi2_contingency

In [52]:
# 카이제곱검정결과 p값이 1로 두변수가 관계가 없음
res = pd.crosstab(train['uenomax'],train['time_m'])
chires = chi2_contingency(res,correction=False)
chires.pvalue

1.0

In [53]:
# chi제곱 검정 결과 uenomax와 관련이 없음
# 드랍
train.drop('time_m', axis=1, inplace=True)
test.drop('time_m', axis=1, inplace=True)

## 전처리 데이터셋 저장

In [ ]:
train.to_csv('Q1_tr_최종전처리.csv', index=False)
test.to_csv('Q1_te_최종전처리.csv', index=False)